In [1]:
debug = 0


import pandas as pd
import sys
import textblob
from tqdm import tqdm,tqdm_pandas

def desc_missing(x):            
    if x['description'].isnull().sum()>0:
        print("Description column has missing values, filling up with недостающий")
        x['description'].fillna("недостающий",inplace=True)
        return x
    else:
        return x
    
def translate(x):
    try:
        return textblob.TextBlob(x).translate(to="en")
    except:
        return x
    
def map_translate(x, cols_translate):
    tqdm.pandas(tqdm())
    for feature in cols_translate:   
        print('>> translating', feature)
        x[feature]=x[feature].progress_map(translate)
        print('done translating', feature)
    return x

def exporter(x,dest_path):
    print("Writting to {}".format(dest_path))
    x.to_csv(dest_path,index=False)
    print("Done")

def read_file(filename):
    if debug==2:
        train_df = pd.read_csv(filename, nrows=10)  
    if debug==1:
        train_df = pd.read_csv(filename, nrows=1000)
    if debug==0:
        train_df = pd.read_csv(filename)          
    return train_df  

In [2]:
filename='../input/train.csv'
dest='../input/translated_train.csv'
print('>> reading...')
train_df = read_file(filename)

>> reading...


In [3]:
# print(train_df.head(5))
train_df.head(5)

,item_id,user_id,region,city,parent_category_name,category_name,param_1,param_2,param_3,title,description,price,item_seq_number,activation_date,user_type,image,image_top_1,deal_probability
0,b912c3c6a6ad,e00f8ff2eaf9,Свердловская область,Екатеринбург,Личные вещи,Товары для детей и игрушки,Постельные принадлежности,NaN,NaN,Кокоби(кокон для сна),"Кокон для сна малыша,пользовались меньше месяц...",400.0,2,2017-03-28,Private,d10c7e016e03247a3bf2d13348fe959fe6f436c1caf64c...,1008.0,0.12789
1,2dac0150717d,39aeb48f0017,Самарская область,Самара,Для дома и дачи,Мебель и интерьер,Другое,NaN,NaN,Стойка для Одежды,"Стойка для одежды, под вешалки. С бутика.",3000.0,19,2017-03-26,Private,79c9392cc51a9c81c6eb91eceb8e552171db39d7142700...,692.0,0.00000
2,ba83aefab5dc,91e2f88dd6e3,Ростовская область,Ростов-на-Дону,Бытовая электроника,Аудио и видео,"Видео, DVD и Blu-ray плееры",NaN,NaN,Philips bluray,"В хорошем состоянии, домашний кинотеатр с blu ...",4000.0,9,2017-03-20,Private,b7f250ee3f39e1fedd77c141f273703f4a9be59db4b48a...,3032.0,0.43177
3,02996f1dd2ea,bf5cccea572d,Татарстан,Набережные Челны,Личные вещи,Товары для детей и игрушки,Автомобильные кресла,NaN,NaN,Автокресло,Продам кресло от0-25кг,2200.0,286,2017-03-25,Company,e6ef97e0725637ea84e3d203e82dadb43ed3cc0a1c8413...,796.0,0.80323
4,7c90be56d2ab,ef50846afc0b,Волгоградская область,Волгоград,Транспорт,Автомобили,С пробегом,ВАЗ (LADA),2110,"ВАЗ 2110, 2003",Все вопросы по телефону.,40000.0,3,2017-03-16,Private,54a687a3a0fc1d68aed99bdaaf551c5c70b761b16fd0a2...,2264.0,0.20797


In [4]:
train_df = desc_missing(train_df)
# train_df.head(5)

In [5]:
train_df.category_name.unique()

array(['Товары для детей и игрушки', 'Мебель и интерьер', 'Аудио и видео',
       'Автомобили', 'Ремонт и строительство',
       'Одежда, обувь, аксессуары', 'Детская одежда и обувь'],
      dtype=object)

In [6]:
# cols_translate = ['parent_category_name', 'category_name', 'param_1', 'param_2', 'param_3', 'title', 'description']

parent_category_name_map = {"Личные вещи" : "Personal belongings",
                            "Для дома и дачи" : "For the home and garden",
                            "Бытовая электроника" : "Consumer electronics",
                            "Недвижимость" : "Real estate",
                            "Хобби и отдых" : "Hobbies & leisure",
                            "Транспорт" : "Transport",
                            "Услуги" : "Services",
                            "Животные" : "Animals",
                            "Для бизнеса" : "For business"}

region_map = {"Свердловская область" : "Sverdlovsk oblast",
            "Самарская область" : "Samara oblast",
            "Ростовская область" : "Rostov oblast",
            "Татарстан" : "Tatarstan",
            "Волгоградская область" : "Volgograd oblast",
            "Нижегородская область" : "Nizhny Novgorod oblast",
            "Пермский край" : "Perm Krai",
            "Оренбургская область" : "Orenburg oblast",
            "Ханты-Мансийский АО" : "Khanty-Mansi Autonomous Okrug",
            "Тюменская область" : "Tyumen oblast",
            "Башкортостан" : "Bashkortostan",
            "Краснодарский край" : "Krasnodar Krai",
            "Новосибирская область" : "Novosibirsk oblast",
            "Омская область" : "Omsk oblast",
            "Белгородская область" : "Belgorod oblast",
            "Челябинская область" : "Chelyabinsk oblast",
            "Воронежская область" : "Voronezh oblast",
            "Кемеровская область" : "Kemerovo oblast",
            "Саратовская область" : "Saratov oblast",
            "Владимирская область" : "Vladimir oblast",
            "Калининградская область" : "Kaliningrad oblast",
            "Красноярский край" : "Krasnoyarsk Krai",
            "Ярославская область" : "Yaroslavl oblast",
            "Удмуртия" : "Udmurtia",
            "Алтайский край" : "Altai Krai",
            "Иркутская область" : "Irkutsk oblast",
            "Ставропольский край" : "Stavropol Krai",
            "Тульская область" : "Tula oblast"}


category_map = {"Одежда, обувь, аксессуары":"Clothing, shoes, accessories",
            "Детская одежда и обувь":"Children's clothing and shoes",
            "Товары для детей и игрушки":"Children's products and toys",
            "Квартиры":"Apartments",
            "Телефоны":"Phones",
            "Мебель и интерьер":"Furniture and interior",
            "Предложение услуг":"Offer services",
            "Автомобили":"Cars",
            "Ремонт и строительство":"Repair and construction",
            "Бытовая техника":"Appliances",
            "Товары для компьютера":"Products for computer",
            "Дома, дачи, коттеджи":"Houses, villas, cottages",
            "Красота и здоровье":"Health and beauty",
            "Аудио и видео":"Audio and video",
            "Спорт и отдых":"Sports and recreation",
            "Коллекционирование":"Collecting",
            "Оборудование для бизнеса":"Equipment for business",
            "Земельные участки":"Land",
            "Часы и украшения":"Watches and jewelry",
            "Книги и журналы":"Books and magazines",
            "Собаки":"Dogs",
            "Игры, приставки и программы":"Games, consoles and software",
            "Другие животные":"Other animals",
            "Велосипеды":"Bikes",
            "Ноутбуки":"Laptops",
            "Кошки":"Cats",
            "Грузовики и спецтехника":"Trucks and buses",
            "Посуда и товары для кухни":"Tableware and goods for kitchen",
            "Растения":"Plants",
            "Планшеты и электронные книги":"Tablets and e-books",
            "Товары для животных":"Pet products",
            "Комнаты":"Room",
            "Фототехника":"Photo",
            "Коммерческая недвижимость":"Commercial property",
            "Гаражи и машиноместа":"Garages and Parking spaces",
            "Музыкальные инструменты":"Musical instruments",
            "Оргтехника и расходники":"Office equipment and consumables",
            "Птицы":"Birds",
            "Продукты питания":"Food",
            "Мотоциклы и мототехника":"Motorcycles and bikes",
            "Настольные компьютеры":"Desktop computers",
            "Аквариум":"Aquarium",
            "Охота и рыбалка":"Hunting and fishing",
            "Билеты и путешествия":"Tickets and travel",
            "Водный транспорт":"Water transport",
            "Готовый бизнес":"Ready business",
            "Недвижимость за рубежом":"Property abroad"}

def map_region_category_to_english(train_df):
    train_df['region_en'] = train_df['region'].apply(lambda x : region_map[x])
    train_df['parent_category_name_en'] = train_df['parent_category_name'].apply(lambda x : parent_category_name_map[x])
    train_df['category_name_en'] = train_df['category_name'].apply(lambda x : category_map[x])
    return train_df

# train_df = map_region_category_to_english(train_df)
# train_df=map_translate(train_df, cols_translate)

In [7]:
train_df.head(5)

,item_id,user_id,region,city,parent_category_name,category_name,param_1,param_2,param_3,title,description,price,item_seq_number,activation_date,user_type,image,image_top_1,deal_probability
0,b912c3c6a6ad,e00f8ff2eaf9,Свердловская область,Екатеринбург,Личные вещи,Товары для детей и игрушки,Постельные принадлежности,NaN,NaN,Кокоби(кокон для сна),"Кокон для сна малыша,пользовались меньше месяц...",400.0,2,2017-03-28,Private,d10c7e016e03247a3bf2d13348fe959fe6f436c1caf64c...,1008.0,0.12789
1,2dac0150717d,39aeb48f0017,Самарская область,Самара,Для дома и дачи,Мебель и интерьер,Другое,NaN,NaN,Стойка для Одежды,"Стойка для одежды, под вешалки. С бутика.",3000.0,19,2017-03-26,Private,79c9392cc51a9c81c6eb91eceb8e552171db39d7142700...,692.0,0.00000
2,ba83aefab5dc,91e2f88dd6e3,Ростовская область,Ростов-на-Дону,Бытовая электроника,Аудио и видео,"Видео, DVD и Blu-ray плееры",NaN,NaN,Philips bluray,"В хорошем состоянии, домашний кинотеатр с blu ...",4000.0,9,2017-03-20,Private,b7f250ee3f39e1fedd77c141f273703f4a9be59db4b48a...,3032.0,0.43177
3,02996f1dd2ea,bf5cccea572d,Татарстан,Набережные Челны,Личные вещи,Товары для детей и игрушки,Автомобильные кресла,NaN,NaN,Автокресло,Продам кресло от0-25кг,2200.0,286,2017-03-25,Company,e6ef97e0725637ea84e3d203e82dadb43ed3cc0a1c8413...,796.0,0.80323
4,7c90be56d2ab,ef50846afc0b,Волгоградская область,Волгоград,Транспорт,Автомобили,С пробегом,ВАЗ (LADA),2110,"ВАЗ 2110, 2003",Все вопросы по телефону.,40000.0,3,2017-03-16,Private,54a687a3a0fc1d68aed99bdaaf551c5c70b761b16fd0a2...,2264.0,0.20797


In [8]:
from googletrans import Translator


print(train_df.category_name.unique())
print(len(train_df.category_name.unique()))

['Товары для детей и игрушки' 'Мебель и интерьер' 'Аудио и видео'
 'Автомобили' 'Ремонт и строительство' 'Одежда, обувь, аксессуары'
 'Детская одежда и обувь']
7


In [9]:
# train_df_translated = train_df

['parent_category_name', 'category_name', 'param_1', 'param_2', 'param_3', 'title', 'description']


TypeError: object of type 'float' has no len()